# 🧠 OmniAGI - Google Colab Edition

**Framework AGI com GPU - Versão Estável**

---

## 1️⃣ Verificar GPU

In [ ]:
import os, sys, torch

IN_COLAB = 'google.colab' in sys.modules
print(f"📍 Ambiente: {'Colab' if IN_COLAB else 'Local'}")

if torch.cuda.is_available():
    GPU_NAME = torch.cuda.get_device_name(0)
    VRAM_GB = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU: {GPU_NAME}")
    print(f"✅ VRAM: {VRAM_GB:.1f} GB")
else:
    VRAM_GB = 0
    print("⚠️ Sem GPU")

## 2️⃣ Instalar Dependências

In [ ]:
if IN_COLAB:
    !pip install -q rwkv structlog chromadb sentence-transformers
    print("✅ Deps instaladas!")
else:
    print("📍 Local - deps já instaladas")

## 3️⃣ Clonar OmniAGI

In [ ]:
import os, sys

if IN_COLAB:
    if not os.path.exists('/content/OmniAGI'):
        !git clone https://github.com/gabrielima7/OmniAGI.git /content/OmniAGI
    OMNIAGI_PATH = '/content/OmniAGI'
    os.chdir(OMNIAGI_PATH)
    !mkdir -p models/rwkv
else:
    OMNIAGI_PATH = '/media/zorin/HD/projetos/OmniAGI'
    os.chdir(OMNIAGI_PATH)

sys.path.insert(0, OMNIAGI_PATH)
print(f"📁 Path: {OMNIAGI_PATH}")

## 4️⃣ Configurar Modelo

⚠️ **IMPORTANTE**: Use modelo 1.6B para evitar crashes!

In [ ]:
import os, torch

# SEMPRE usar 1.6B no Colab para estabilidade
# O modelo 3B causa crash no kernel
MODEL_SIZE = "1b6"
MODEL_PATH = "models/rwkv/rwkv-6-1b6.pth"
MODEL_URL = "https://huggingface.co/BlinkDL/rwkv-6-world/resolve/main/RWKV-x060-World-1B6-v2.1-20240328-ctx4096.pth"
STRATEGY = "cuda fp16"  # 1.6B cabe fácil na GPU

print(f"📦 Modelo: RWKV-6 {MODEL_SIZE.upper()}")
print(f"⚙️ Estratégia: {STRATEGY}")
print(f"📁 Existe: {os.path.exists(MODEL_PATH)}")

## 5️⃣ Download do Modelo

In [ ]:
import os

if not os.path.exists(MODEL_PATH):
    print(f"⬇️ Baixando RWKV {MODEL_SIZE.upper()}...")
    os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
    !wget -q --show-progress -O {MODEL_PATH} {MODEL_URL}
    print("✅ Download concluído!")
else:
    print(f"✅ Modelo existe: {MODEL_PATH}")

print(f"📊 Tamanho: {os.path.getsize(MODEL_PATH) / 1e9:.2f} GB")

## 6️⃣ Carregar Modelo

In [ ]:
import torch
from rwkv.model import RWKV
from rwkv.utils import PIPELINE, PIPELINE_ARGS

# Limpar cache GPU antes de carregar
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print(f"🔄 Carregando RWKV-6 {MODEL_SIZE.upper()}...")
model = RWKV(model=MODEL_PATH, strategy=STRATEGY)
pipeline = PIPELINE(model, 'rwkv_vocab_v20230424')
args = PIPELINE_ARGS(temperature=0.7, top_p=0.9)

print("✅ Modelo carregado!")
if torch.cuda.is_available():
    used = torch.cuda.memory_allocated() / 1e9
    print(f"📊 VRAM: {used:.1f} GB")

## 7️⃣ Testar Geração

In [ ]:
import time

prompt = "Artificial General Intelligence is"
print(f"📝 Prompt: {prompt}")

start = time.time()
output = pipeline.generate(prompt, token_count=50, args=args)
elapsed = time.time() - start

print(f"\n🤖 Output: {output}")
print(f"\n⚡ {50/elapsed:.1f} tokens/s")

## 8️⃣ Testar Módulos OmniAGI

In [ ]:
print("🧪 TESTANDO MÓDULOS")
print("=" * 40)

# Consciousness
try:
    from omniagi.consciousness import ConsciousnessEngine
    c = ConsciousnessEngine()
    c.awaken()
    print(f"🧠 Consciousness: {c.reflect()['state']} ✅")
except Exception as e:
    print(f"🧠 Consciousness: {e} ❌")

# Math
try:
    from omniagi.benchmarks.arc_solver import ChainOfThoughtSolver
    s = ChainOfThoughtSolver()
    r = s.solve('sum', '25+37')
    print(f"📐 Math 25+37={r.answer} {'✅' if r.answer=='62' else '❌'}")
except Exception as e:
    print(f"📐 Math: {e} ❌")

# RAG
try:
    from omniagi.memory.rag import RAGSystem
    rag = RAGSystem('test')
    rag.initialize()
    print(f"📚 RAG: OK ✅")
except Exception as e:
    print(f"📚 RAG: {e} ❌")

print("=" * 40)

## 9️⃣ Chat com AGI

In [ ]:
def chat(prompt, tokens=80):
    p = f"User: {prompt}\nAssistant:"
    r = pipeline.generate(p, token_count=tokens, args=args)
    return r.split("User:")[0].strip() if "User:" in r else r.strip()

print("🤖 Chat")
for q in ["What is AGI?", "Calculate 15*3"]:
    print(f"\n👤 {q}")
    print(f"🤖 {chat(q)[:150]}")

## 🔟 Sua Pergunta

In [ ]:
pergunta = "What is consciousness?"

print(f"👤 {pergunta}")
print(f"\n🤖 {chat(pergunta, tokens=100)}")

---

**GitHub**: https://github.com/gabrielima7/OmniAGI

🌟 **OmniAGI** 🌟